In [ ]:
import vk
import pandas as pd
import time
from datetime import datetime

In [ ]:
# opennig vk session with access token.
session = vk.Session(access_token='0dfed272d8c2c0882f26fabaee86c1c23eca901348c1e8476f93a1fd83fe06b4b9fc4070f7c0f1aa269f6')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

In [ ]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [ ]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

In [ ]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [ ]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

In [ ]:
key_words = ['#глаза', 
             '#глазки', '#зрение', '#лечениезрения', '#детскоезрение', '#лечениеглаз',
             '#офтальмолог','#офтальмология', '#офтальмохирург', '#офтальмохирургия', '#детскоезрение',
             '#детскийофтальмолог', '#офтальмологмосква','#дакриоцистит', '#дакриоциститноворожденных',
             '#дакриоциститмассаж','#дакриоциститлечение', '#дакриоцисториностомия', '#зондированиеносослёзногоканала',
             '#зондированиеноворождённых','#нистагм', '#лечениенистагма','#нистагмоперация', '#нистагмъ',
             '#близорукость', '#миопия', '#лечениеблизорукости', '#близорукостьнеприговор',
             '#близоркостьлечение', '#близорукостьонатакая', '#близорукостьнепорок',
             '#близорукостьудетей','#дальнозоркость', '#гиперметропия', '#лечениедальнозоркости',
             '#дальнозоркостьлечение','#дальнозоркостьудетей', '#дальнозоркости','#косоглазие',
             '#страбизм', '#лечениекосоглазия', '#косоглазиелечение', '#косоглазиеэтосексуально',
             '#косоглазиебишкек', '#косоглазиенепорок', '#косоглазие дети', '#косоглазиеэтомодно',
             '#астигматизм', '#астигматизмлечение', '#астигматизмудетей','#амблиопия', '#амблиопиявысокойстепени',
             '#амблиопиямосква', '#катаракта','#катарактавладивосток', '#катарактачелябинск',
             '#катаракта лечение москва','#катарактаоперация', '#глаукома', '#глаукомалечение', 
             '#глаукомавладивосток','#глаукомачелябинск', '#птоз', '#птозверхнеговека', '#птозвека',
             '#птозвек', '#птозанет','#лазернаякоррекция', '#лазернаякоррекциязрения', '#лазернаякоррекциязрениямосква',
             '#лазернаякоррекцияакция', '#лазернаякоррекциямосква', '#лазернаякоррекциязренияомикрон',
             '#лазернаякоррекцияскидки', '#lasik', '#фрк', '#smile', '#ласик', '#ласикмосква', '#ячмень', '#халязион']
desired_time = 86400*365 # how much back we want to go in seconds
entire_df = pd.DataFrame()
k = 1
for key_word in key_words:
    print('search for keyword: ', key_word)
    date = time.time() # now
    excess_time = True
    all_df = pd.DataFrame()
    c=1
    while excess_time:
        if k%60 == 0:
            print('Rest for 10 min')
            time.sleep(600)
            print('Work')
         # print iteration count, extended tells us group vs user
        g=api.newsfeed.search(q=[key_word],count=200, end_time = date-1, start_time=1,extended=1)
        if len(g)>1: # check if g contains data
            all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
            date=g[-1]['date']
        elif len(g) <= 1 and c == 1: # check if there is data
            print('No posts with that keyword(s)! and FUCK YOU!!!!')
            time.sleep(0.4)
            break
        elif len(g) <= 1 and c != 1: # check if there are posts eft to scrape
            print('No more posts to scrape!')
            break

        if date<time.time()-desired_time: # check termination condition
            excess_time = False
        print(c,'iteration')
        c+=1
        k+=1
    all_df['keyword'] = key_word
    print(len(all_df))
    entire_df = pd.concat([entire_df,all_df])
'''    print ('tasks done, now sleeping for 30 seconds')
    for i in range(30,0,-1):
        time.sleep(1)
        print (i)'''
print(len(entire_df))

In [ ]:
entire_df.to_csv(r'D:\vk_all_keywords_365_days.csv')

In [ ]:
all_id = pd.DataFrame()
all_df_new_2=entire_df[entire_df.owner_id>0].reset_index()
unique_users = list(set(all_df_new_2['owner_id'].astype(int)))
for part in range(int(len(unique_users)/9000)+1):
    
    # take all positive ids ( user ids , negative ids are group ids)
    
    all_df_new=unique_users[(part*9000):(1+part)*9000]
    
    for user_part in range(int(len(all_df_new)/901)+1):
    # get all users
        all_id_2 = pd.DataFrame(api.users.get(user_ids=all_df_new[(user_part*901):(1+user_part)*901], 
                                              fields = ['occupation','education',
                                                        'universities','bdate',
                                                        'city','interests',
                                                        'activities','site']))
        all_id = pd.concat([all_id,all_id_2])
        time.sleep(0.3)

    
try:     
    # fill active users status with valid
    all_id['deactivated'].fillna('Valid',inplace=True) 
    # select all deactivated user ids
    deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

    # select all active user posts
    active_users = all_df_new_2[-all_df_new_2.owner_id.isin(deactivated_ids)]
    active_users_info = all_id[all_id['deactivated']=='Valid']
except:
    active_users = all_df_new_2  
    active_users_info = all_id
# merge post info and user info
overall_data = active_users.merge(active_users_info, left_on='owner_id', right_on='uid')

In [ ]:
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date','key_word'])
# resetting the index it make circles of 0-200
#active_users.reset_index(inplace = True)
for i in tqdm(range(len(overall_data))):    
    new_df.loc[i,'key_word'] = overall_data.loc[i]['keyword']
    new_df.loc[i,'user_id'] = overall_data.user.loc[i]['uid']
    new_df.loc[i,'sex'] = overall_data.user.loc[i]['sex']   
    new_df.loc[i,'Name_Surname'] = overall_data.user.loc[i]['first_name']+'_'+overall_data.user.loc[i]['last_name']
    new_df.loc[i,'post'] = overall_data.text.loc[i]
    new_df.loc[i,'likes_count'] = overall_data.likes.loc[i]['count']
    new_df.loc[i,'comments_count'] = overall_data.comments.loc[i]['count'] 
    new_df.loc[i,'reposts_count'] = overall_data.reposts.loc[i]['count']
    try:
        new_df.loc[i,'platform'] = overall_data.post_source.loc[i]['type']+'_'+overall_data.post_source.loc[i]['platform']
    except:
        new_df.loc[i,'platform'] = overall_data.post_source.loc[i]['type']
    new_df.loc[i,'date'] = datetime.fromtimestamp(overall_data.date.loc[i]).strftime('%Y-%m-%d %H:%M:%S')
new_df['sex'] = new_df['sex'].apply(lambda x: 'Female' if x==1 else 'Male' if x==2  else 'Not Specified')
#new_df.head()

In [ ]:
# getting the comments of posts
def commentscrapper(x):
    uid = x['user_id']
    key_word = x['key_word']
    post = api.wall.search(owner_id = uid, query = key_word)
    time.sleep(0.4)
    if post[0] == 1:
        pid = post[1]['id']
        likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
        time.sleep(0.4)
        reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
        time.sleep(0.4)
        try:           
            coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
            return {'comments':[post[0],coms[1:]],'users_liked': likes,'users_reposted': reposts}
        except:
            return {'comments':[post[0],'Access Denied'],'users_liked':likes,'users_reposted': reposts}
    elif post[0]==0:
        return {'comments':'post not found','users_liked':'post not found','users_reposted': 'post not found'}
    else:
        try:
            my_date = x['date']
            post_dates = [datetime.fromtimestamp(y['date']).strftime('%Y-%m-%d %H:%M:%S') for y in post[1:]]
            index = post_dates.index(my_date)
            try:
                pid = post[index+1]['id']
                likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
                time.sleep(0.4)
                reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
                time.sleep(0.4)
                coms = api.wall.getComments(owner_id = uid, post_id = pid, need_likes = 1)
                return {'comments':[post[0],coms[1:]],'users_liked':likes,'users_reposted': reposts}
            except:
                pid = post[index+1]['id']
                likes = api.likes.getList(type='post', item_id = pid, owner_id = uid)
                time.sleep(0.4)
                reposts = api.likes.getList(type='post', item_id = pid, owner_id = uid, filter = 'copies')
                time.sleep(0.4)
                return {'comments':[post[0],'Access Denied'],'users_liked':likes,'users_reposted': reposts}
        except:
            return {'comments':'time not in Index','users_liked':'time not in Index','users_reposted': 'time not in Index'}
    time.sleep(0.4)

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm.pandas(tqdm())
more_new_df = new_df.progress_apply(commentscrapper,axis = 1)
comments_likes_reposts = more_new_df.apply(pd.Series)

In [ ]:
final_data = pd.concat([overall_data,comments_likes_reposts],axis = 1)

In [ ]:
final_data.to_csv(r'D:\vk_all_keywords_365_days_pretty.csv')

In [ ]:
short_df = final_data.copy()

In [ ]:
pd.options.display.max_columns = 45
short_df.head()

In [ ]:
# remove posts with more than 300 characters and posts with no characters
short_df['post']=short_df.text.apply(lambda x: x if len(x)<300 and len(x) !=0 else 'Too Long')

In [ ]:
not_long = short_df[short_df['post']!='Too Long'].reset_index().drop('index', axis = 1)

In [ ]:
import re
list_of_animals = 'кот|кошка|котёнок|котяра|котяры|котяр|котище|котик|собака|пес|собачка|кобель|псина|сука|птица|пташка|птенцы|птенец|щенок'
def find_animal(x):
    my_list = re.findall(list_of_animals,x)
    if len(my_list) == 0:
        return(True)
    else:
        return(False)

In [ ]:
short_df['animal'] = short_df.post.apply(lambda x: True if find_animal(x)==True else False)

In [ ]:
short_df.to_csv(r'D:\vk_all_keywords_365_days_short.csv')